# Data Collection

This notebook aims to collect historical price data for stocks in the S&P 500 from the **yfinance** module.

In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import statsmodels.tsa.vector_ar as ar
import datetime
from dateutil.relativedelta import relativedelta
import itertools
import math

Get symbols of Current Stocks in S&P500 and download data

In [11]:
payload = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = payload[0]
symbols = first_table['Symbol'].values.tolist()

# Update symbols to correct for yfinance accessibility
symbols = list(map(lambda x: x.replace('BRK.B', 'BRK-B'), symbols))
symbols = list(map(lambda x: x.replace('BF.B', 'BF-B'), symbols))

symbols

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',


In [12]:
# Download historical stock data based on the start and end dates
data = yf.download(symbols, start = "2015-01-01", end = "2025-01-01", interval="1d")

[*********************100%***********************]  503 of 503 completed


Clean data for any timeseries that was missing

In [13]:
close_data = data['Close']

close_data.head()

Ticker,A,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.353008,24.320433,43.156219,NaN,36.915020,18.539352,75.312714,72.339996,44.890644,38.700108,...,100.853874,24.115881,126.880699,26.153711,59.510235,33.441818,42.833309,100.930527,77.430000,40.157524
2015-01-05,36.653099,23.635288,42.344048,NaN,36.923252,18.428413,74.041138,71.980003,44.074306,37.361015,...,100.129463,24.115881,124.721306,25.856846,57.881931,31.360485,41.963013,104.695580,76.339996,39.916462
2015-01-06,36.082130,23.637508,42.134449,NaN,36.503944,18.469618,73.507057,70.529999,43.039742,36.624493,...,99.631401,24.089176,122.060822,25.994427,57.574223,31.176062,41.447945,103.808128,75.790001,39.527027
2015-01-07,36.561012,23.968962,43.837395,NaN,36.799923,18.577387,75.049934,71.110001,43.492367,37.175014,...,101.057640,24.162630,125.645554,26.218882,58.157585,31.421951,42.821457,106.389877,77.720001,40.342972
2015-01-08,37.656925,24.889904,44.295860,NaN,37.556316,18.900694,76.194389,72.919998,44.260193,36.631947,...,102.212204,24.423092,127.157173,26.522999,59.125591,31.659071,43.567413,107.519394,79.379997,40.964207


In [14]:
close_data.dropna(axis=1, inplace=True)

close_data.head()

C:\Users\luoli\AppData\Local\Temp\ipykernel_34248\561113609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_data.dropna(axis=1, inplace=True)


Ticker,A,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,ADP,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,37.353008,24.320433,43.156219,36.915020,18.539352,75.312714,72.339996,44.890644,38.700108,66.660583,...,100.853874,24.115881,126.880699,26.153711,59.510235,33.441818,42.833309,100.930527,77.430000,40.157524
2015-01-05,36.653099,23.635288,42.344048,36.923252,18.428413,74.041138,71.980003,44.074306,37.361015,66.612518,...,100.129463,24.115881,124.721306,25.856846,57.881931,31.360485,41.963013,104.695580,76.339996,39.916462
2015-01-06,36.082130,23.637508,42.134449,36.503944,18.469618,73.507057,70.529999,43.039742,36.624493,66.060287,...,99.631401,24.089176,122.060822,25.994427,57.574223,31.176062,41.447945,103.808128,75.790001,39.527027
2015-01-07,36.561012,23.968962,43.837395,36.799923,18.577387,75.049934,71.110001,43.492367,37.175014,66.636551,...,101.057640,24.162630,125.645554,26.218882,58.157585,31.421951,42.821457,106.389877,77.720001,40.342972
2015-01-08,37.656925,24.889904,44.295860,37.556316,18.900694,76.194389,72.919998,44.260193,36.631947,68.165184,...,102.212204,24.423092,127.157173,26.522999,59.125591,31.659071,43.567413,107.519394,79.379997,40.964207


In [15]:
close_data.to_csv('./data/Data.csv')

In [17]:
stock_tickers = close_data.columns

stock_tickers

Index(['A', 'AAPL', 'ABBV', 'ABT', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP',
       ...
       'WTW', 'WY', 'WYNN', 'XEL', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZTS'],
      dtype='object', name='Ticker', length=470)